<h1 style="text-align:center;">Nerve Quadrature</h1>
<h3 style="text-align:center;margin:0;">Vijay Stroup</h3>
<p style="text-align:center;margin:0;">University of Central Florida • November 6, 2021</p>

# Introduction

Consider the function $\psi: \mathbb{R}\rightarrow\mathbb{R}$

$$
\psi(x) =
\frac{1}2 + \frac{1}\pi \int_0^\infty \frac{A(s)sin(sx)}{s(A(s)^2 + c^2s^2)} ds +
\frac{c}\pi \int_0^\infty \frac{cos(sx)}{A(s)^2 + c^2s^2} ds
$$

$$ A(s) = 1 + 2(1 - cos(s)) $$

<br/><br/>

$\psi$ is the wave function representation of a impuluse in the human nervious system. We can use different methods of integration
to estimate $\psi$ at different values of x and c.  
The methods of quadrature that will be used in this project will be the trapezoidal and simpson methods. We can then use
Richardson's error estimate to determine how accurate each of the methods in this project are. The reason of using Richardson's
error estimate is because we do not have any true values to compare to.

I believe that Simpon's Rule will be more accurate when estimaing what this function should be. It will also be able to be more accurate with many less nodes than required by the trapezodial rule. This is shown by calculating the number of nodes needed to achieve a certain error for both methods.

In [12]:
from scipy.integrate import trapezoid, simpson

f = lambda x: x**2

xs = [0, 1, 2, 3, 4, 5]
ys = [f(i) for i in xs]
print(trapezoid(ys, xs))
print(simpson(ys, xs))

42.5
41.83333333333333
